In [1]:
import sentier_data_tools as sdt
import sentier_peakachu as sp
from datetime import datetime

In [2]:
sp.create_local_electricity_datastorage()

/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/utils_location.py:40: UserWarning: ISO code ABW not found
  warnings.warn(f"ISO code {iso_code} not found")
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:118: UserWarning: Location not found for ABW, skipping Dataset creation
  warnings.warn(f"Location not found for {country}, skipping Dataset creation")
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/utils_location.py:40: UserWarning: ISO code BES not found
  warnings.warn(f"ISO code {iso_code} not found")
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:118: UserWarning: Location not found for BES, skipping Dataset creation
  warnings.warn(f"Location not found for {country}, skipping Dataset creation")
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/utils_location.py:40: UserWarning: ISO code BMU not found
  warnings.warn(f"ISO code {iso_code} not found")
/Users

## ElectricitySourceModel

In [3]:
el_from_gas = sdt.ProductIRI("https://example.com/gas")
location = sdt.GeonamesIRI(sp.get_geonames_iri_from_iso_code("DE"))
demand = sdt.Demand(
    product_iri=el_from_gas,
    unit_iri=sdt.UnitIRI("https://vocab.sentier.dev/units/unit/KilowattHour"),
    amount=10,
    spatial_context=location,
    begin_date=datetime(2022, 10, 8, 6, 49, 0),
    end_date=datetime(2022, 10, 9, 6, 59, 0),
)
model = sp.ElectricitySourceModel(demand=demand, run_config=sdt.RunConfig())
plant_emissions = model.get_model_data()
model.calculate_average_source_emission_factor(plant_emissions)

0.5459558400294707

## ElectricityMixModel

In [22]:
generic_electricity = sdt.ProductIRI("http://openenergy-platform.org/ontology/oeo/OEO_00000139")
location = sdt.GeonamesIRI(sp.get_geonames_iri_from_iso_code("DE"))
demand = sdt.Demand(
    product_iri=generic_electricity,
    unit_iri=sdt.UnitIRI("https://vocab.sentier.dev/units/unit/KilowattHour"),
    amount=10,
    spatial_context=location,
    begin_date=datetime(2022, 10, 8, 15, 46, 0),
    end_date=datetime(2022, 10, 8, 15, 49, 0),
)
model = sp.ElectricityMixModel(demand=demand, run_config=sdt.RunConfig())
df = model.get_model_data()

In [23]:
mix = model.calculate_market_mix(df)

In [25]:
model.calculate_impact(mix)

2024-10-11 00:11:29.858 | DEBUG    | sentier_peakachu.model:calculate_impact:106 - Market shares: https://example.com/coa    0.865719
https://example.com/gas    0.134281
dtype: object
2024-10-11 00:11:29.859 | DEBUG    | sentier_peakachu.model:calculate_impact:107 - Emission factors: [1.0415439556617119, 0.5459558400294707]


9.749957884757078

## Other

In [10]:
import requests

def get_geoname_info(geoname_id, username):
    url = f"http://api.geonames.org/getJSON?geonameId={geoname_id}&username={username}"
    try:
        response = requests.get(url)
        # Check HTTP status code for response received successfully
        if response.status_code == 200:
            data = response.json()
            # Check if the data contains any meaningful information
            if data:
                # Check if there is an error message in the data
                if 'status' in data:
                    return {'error': data['status']['message']}
                else:
                    # Here, you can add more checks depending on your needs
                    return {'data': data}
            else:
                return {'error': 'No data returned from GeoNames API'}
        else:
            return {'error': f'HTTP Error {response.status_code}'}
    except requests.exceptions.RequestException as e:
        # Handle connection errors
        return {'error': str(e)}

# Example usage
geoname_id = 2921044  # Example ID for Germany
username = 'diepers'  # Replace with your GeoNames username
result = get_geoname_info(geoname_id, username)
if 'error' in result:
    print("Error:", result['error'])
else:
    if result['data'].get("fcl") == 'A' and result['data'].get("fcode") == 'PCLI':
        print("The ID represents a country.")
    else:
        print("The ID does not represent a country.")


Error: HTTP Error 401


In [3]:
import warnings
import polars as pl

BASE_URL = "https://sws.geonames.org/"
LOOKUP_FILE = "../data/iso_iri_table.csv"
LOOKUP_SEPARATOR = "\t"

In [8]:

iso_code = "FAS"
if len(iso_code) not in [2, 3]:
    raise ValueError("Please provide a two-digit or three-digit ISO code")

lookup = pl.scan_csv(LOOKUP_FILE, separator=LOOKUP_SEPARATOR)
column = "ISO3" if len(iso_code) == 3 else "ISO2"
query = f"SELECT geonameid FROM self WHERE {column} = '{iso_code}'"

In [10]:
geoname_id = lookup.sql(query).collect()

In [14]:
geoname_id.is_empty()

True